#### Score: 28.8942

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy import stats

Using TensorFlow backend.
E:\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1

In [2]:
X_train = pd.read_csv('data_dengue/dengue_features_train.csv', index_col=None)
df_y_train = pd.read_csv('data_dengue/dengue_labels_train.csv', index_col=None)
X_test = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)

In [3]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000
X_train_new = X_train.fillna(X_train.mean())
X_train_sj=X_train_new[X_train_new.city=='sj']
X_train_iq=X_train_new[X_train_new.city=='iq']
X_train_sj = X_train_sj.drop(['city'], axis=1)
X_train_iq = X_train_iq.drop(['city'], axis=1)

In [4]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
X_test_new = X_test.fillna(X_test.mean())
X_test_sj=X_test_new[X_test_new.city=='sj']
X_test_iq=X_test_new[X_test_new.city=='iq']
X_test_sj = X_test_sj.drop(['city'], axis=1)
X_test_iq = X_test_iq.drop(['city'], axis=1)

In [5]:
X_train_sj_float = X_train_sj.astype('float64')
X_train_iq_float = X_train_iq.astype('float64')
X_test_sj_float = X_test_sj.astype('float64')
X_test_iq_float = X_test_iq.astype('float64')

In [6]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_sj_scaled = min_max_scaler.fit_transform(X_train_sj_float)
X_train_iq_scaled = min_max_scaler.fit_transform(X_train_iq_float)
X_test_sj_scaled = min_max_scaler.transform(X_test_sj_float)
X_test_iq_scaled = min_max_scaler.transform(X_test_iq_float)

In [7]:
print(X_train_sj_scaled[:5, :])
print(X_train_sj_scaled.shape)

[[0.         0.32692308 0.         0.58783972 0.62676332 0.52370092
  0.54188858 0.03179724 0.26100844 0.26918536 0.33993342 0.30769231
  0.45205479 0.05609115 0.3181382  0.03179724 0.29739227 0.35849109
  0.35968379 0.4403183  0.30337079 0.28205128 0.05230467]
 [0.         0.34615385 0.00106594 0.64041572 0.66981079 0.43529974
  0.49214226 0.05842294 0.36299338 0.38488784 0.5284738  0.47692308
  0.52054795 0.0314461  0.51021387 0.05842294 0.47346033 0.28598728
  0.53557312 0.34217507 0.56179775 0.56410256 0.02811376]
 [0.         0.36538462 0.00213187 0.48741177 0.70428426 0.42268026
  0.52666235 0.08842806 0.45402692 0.45690673 0.7103557  0.41538462
  0.64383562 0.04574934 0.73498766 0.08842806 0.66450897 0.26584733
  0.53557312 0.36339523 0.61797753 0.64102564 0.13533835]
 [0.         0.38461538 0.00319781 0.59454599 0.78500526 0.59484481
  0.67286631 0.03932412 0.48688113 0.51475797 0.69511127 0.55384615
  0.60273973 0.02436459 0.65265972 0.03932412 0.64176068 0.30209923
  0.640316

In [8]:
train_1s_sj = X_train_sj_scaled.reshape(X_train_sj_scaled.shape[0],1,X_train_sj_scaled.shape[1])
train_1s_iq = X_train_iq_scaled.reshape(X_train_iq_scaled.shape[0],1,X_train_iq_scaled.shape[1])

In [9]:
train_1s_sj.shape

(936, 1, 23)

In [10]:
data_y_1s_sj = np.array(df_y_train[df_y_train.city=='sj']["total_cases"])
data_y_1s_iq = np.array(df_y_train[df_y_train.city=='iq']["total_cases"])
data_y_1s_sj.shape

(936,)

In [11]:
model_1s_sj = Sequential()
model_1s_sj.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23))))
model_1s_sj.add(Bidirectional(LSTM(40, activation='relu', return_sequences=True)))
model_1s_sj.add(Bidirectional(LSTM(20, activation='relu')))
model_1s_sj.add(Dense(1))
model_1s_sj.compile(optimizer='adam', loss='mae')

In [12]:
%%time
model_1s_sj.fit(train_1s_sj, data_y_1s_sj, epochs=400, verbose=1)


Epoch 1/400
936/936 [==============================] - 7s 7ms/step - loss: 34.1011
Epoch 2/400
936/936 [==============================] - 0s 408us/step - loss: 30.6186
Epoch 3/400
936/936 [==============================] - 0s 431us/step - loss: 25.3261
Epoch 4/400
936/936 [==============================] - 0s 399us/step - loss: 24.8015
Epoch 5/400
936/936 [==============================] - 0s 416us/step - loss: 24.3795
Epoch 6/400
936/936 [==============================] - 0s 424us/step - loss: 24.0856
Epoch 7/400
936/936 [==============================] - 0s 382us/step - loss: 24.0449
Epoch 8/400
936/936 [==============================] - 0s 404us/step - loss: 23.6680
Epoch 9/400
936/936 [==============================] - 0s 404us/step - loss: 23.4376
Epoch 10/400
936/936 [==============================] - 0s 413us/step - loss: 23.2761
Epoch 11/400
936/936 [==============================] - 0s 395us/step - loss: 23.2689
Epoch 12/400
936/936 [==============================] - 0s 462us

E:\Miniconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.150934). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


936/936 [==============================] - 1s 591us/step - loss: 19.6659
Epoch 149/400
936/936 [==============================] - 0s 358us/step - loss: 19.5003
Epoch 150/400
936/936 [==============================] - 0s 367us/step - loss: 19.5992
Epoch 151/400
936/936 [==============================] - 0s 366us/step - loss: 19.7144
Epoch 152/400
936/936 [==============================] - 0s 360us/step - loss: 19.5337
Epoch 153/400
936/936 [==============================] - 0s 377us/step - loss: 19.7453
Epoch 154/400
936/936 [==============================] - 0s 361us/step - loss: 19.6432
Epoch 155/400
936/936 [==============================] - 0s 378us/step - loss: 19.6798
Epoch 156/400
936/936 [==============================] - 0s 405us/step - loss: 19.7481
Epoch 157/400
936/936 [==============================] - 1s 542us/step - loss: 19.5738
Epoch 158/400
936/936 [==============================] - 0s 421us/step - loss: 19.5839
Epoch 159/400
936/936 [==============================] - 

In [13]:
model_1s_iq = Sequential()
model_1s_iq.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23))))
model_1s_iq.add(Bidirectional(LSTM(40, activation='relu', return_sequences=True)))
model_1s_iq.add(Bidirectional(LSTM(20, activation='relu')))
model_1s_iq.add(Dense(1))
model_1s_iq.compile(optimizer='adam', loss='mae')

In [14]:
%%time
model_1s_iq.fit(train_1s_iq, data_y_1s_iq, epochs=400, verbose=0)

Wall time: 1min 16s


In [15]:
data_1s_test_sj = X_test_sj_scaled.reshape(X_test_sj_scaled.shape[0],1,X_test_sj_scaled.shape[1])
data_1s_test_iq = X_test_iq_scaled.reshape(X_test_iq_scaled.shape[0],1,X_test_iq_scaled.shape[1])
data_1s_test_sj.shape

(260, 1, 23)

In [16]:
predict_sj = model_1s_sj.predict(data_1s_test_sj)

In [17]:
predict_iq = model_1s_iq.predict(data_1s_test_iq)

In [18]:
X_test_predict = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)
result_columns_sj = X_test[X_test.city=='sj'][['city','year','weekofyear']]
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_df_sj = pd.concat([result_columns_sj, result_cases_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,30
1,sj,2008,19,11
2,sj,2008,20,14
3,sj,2008,21,15
4,sj,2008,22,11


In [19]:
result_columns_iq = X_test[X_test.city=='iq'][['city','year','weekofyear']]
result_cases_iq = pd.DataFrame(predict_iq)
result_cases_iq.columns = ['total_cases']
result_columns_iq = result_columns_iq.reset_index()
result_df_iq = pd.concat([result_columns_iq, result_cases_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq = result_df_iq.drop(['index'], axis=1)
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,2
1,iq,2010,27,1
2,iq,2010,28,5
3,iq,2010,29,0
4,iq,2010,30,3


In [20]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)
result.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,30
1,sj,2008,19,11
2,sj,2008,20,14
3,sj,2008,21,15
4,sj,2008,22,11


In [21]:
result.to_csv('lstm_2_city_BI_50_40_30.csv', index=False)